In [1]:
import sys, os
sys.path.append('..')
hls4ml_catapult_path = '/home/manuelbv/hls4ml_catapult'
sys.path = [hls4ml_catapult_path] + sys.path

# Import datetime to get today's date
from datetime import datetime

""" Let's add our custom netsurf code """
import netsurf

""" Get netsurf path """
import os 
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath(netsurf.__file__)))

Adding /mnt/raid0/asic/projects/NU/netsurf/manuelbv to sys.path
[INFO] - Added qkeras to sys.path from /mnt/raid0/asic/projects/NU/netsurf/manuelbv/qkeras
[INFO] - Loaded theme: default
                   ▗▙▝ ▖▗  ▜▛ ▜  ▜ 
   𐌍 ↠⌾↞ 𐌃 𐌖 𐌔     ▖▘▙ ▝▜▘▗▛   ▝▙  
                   ▗▖   ▜ ▜▛    ▙▜ 

Logging to file: /home/manuelbv/.nodus/nodus_20250416_103949.log


-- Date: 16/Apr/2025
╭───────┬─────────────╮
╰ INFO ─┤ 10:39:49.94 │ - Nodus initialized
        │ 10:39:49.94 │ - Nodus version: 0.1.0
        │ 10:39:49.94 │ - Nodus imported
        │ 10:39:49.94 │ - Jobs imported
        │ 10:39:49.94 │ - JobManager imported
        │ 10:39:49.94 │ - Nodus ready to use


Found config file: /home/manuelbv/.netsurf/config


        │ 10:39:50.16 │ - Created jobs table in NodusDB instance 'netsurf_db'
        │ 10:39:50.16 │ - Created job_dependencies table in NodusDB instance 'netsurf_db'
        │ 10:39:50.16 │ - Added NodusDB instance 'netsurf_db' linked to database 'netsurf_db'


In [ ]:
import pergamos as pg

benchmark_name = 'keyword_spotting'
# Set variables
qscheme = "q<6,2,1>"
pruning = 0.125
prerank = True


""" First of all, let's define a quantization Scheme """
Q = netsurf.QuantizationScheme(qscheme)
print(Q)

# Set filename
benchmarks_dir = os.path.join(parent_dir, 'benchmarks')
datasets_dir = os.path.join(parent_dir, 'datasets')

filename = f"7_HLS_{Q._scheme_str.no_special_chars()}_bmark_{benchmark_name}_prune{str(pruning).replace('.','_')}.html"
print(filename)
doc = pg.Document(filename, theme="default")
doc.required_scripts.add('mathjax')


""" Add a title to the document """
doc.append(pg.Markdown(f"""# Benchmarks Quantization Assertion
> Author: Manuel B Valentin

> Creation date: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

> Project: netsurf

> Used packages: netsurf, tensorflow, numpy, matplotlib, pergamos
        
"""))

# Add a tab container
tabs = pg.TabbedContainer({'Motivation': [], 
                        'Pre-training analysis': [],
                        'Training': [],
                            'Post-Training': [],
                        'BER Injection': [],
                        'Conclusions': []})

# Get individual tabs
tabmotivation = tabs['Motivation']
tabpretraining = tabs['Pre-training analysis']
tabtraining = tabs['Training']
tabposttraining = tabs['Post-Training']
tabber = tabs['BER Injection']
tabconclusions = tabs['Conclusions']

# Add to documnt
doc.append(tabs)

# Add a markdown description of what we want to achieve with this report in the first tab
md = r"""
## 1. Loss Taylor expansion

Given a loss function $\mathcal{L}(w)$, where $w$ is the vector of all weights in the network, the Taylor expansion around some point $w_0$ (say, the trained weights) for a small perturbation $\Delta w$ is:

$$\mathcal{L}(w_0 + \Delta w) \approx \mathcal{L}(w_0) + \nabla \mathcal{L}(w_0)^T \Delta w + \frac{1}{2} \Delta w^T H \Delta w$$

Where:

* $\nabla \mathcal{L}(w_0)$ is the gradient vector of the loss at w_0
* $H$ is the Hessian matrix, i.e. $H = \nabla^2 \mathcal{L}(w_0)$

---

## 2. If the model is trained??

If the model has been well trained, then:

$\nabla \mathcal{L}(w_0) \approx 0$

Because you're sitting near a (local) minimum.

This removes the linear term:
$\mathcal{L}(w_0 + \Delta w) - \mathcal{L}(w_0) \approx \frac{1}{2} \Delta w^T H \Delta w$

So the change in loss caused by a perturbation $\Delta w$ is approximately:
$\Delta \mathcal{L} \approx \frac{1}{2} \Delta w^T H \Delta w$

---

## 3. Interpretation for bit flips

A bit flip in the quantized weights causes a small but structured change in the weights:

* Say, flipping the 3rd bit in weight $w_i$ causes it to change by $\delta_i$, so:

$\Delta w = \begin{bmatrix}
0 \\ \cdots \\ \delta_i \\ \cdots \\ 0
\end{bmatrix}$

Then the loss increase is (approximately):
$\Delta \mathcal{L} \approx \frac{1}{2} \delta_i^2 H_{ii}$

If multiple bits are flipped across weights, you sum their pairwise interactions via H, including off-diagonal terms (if not ignored).

--- 

## 4. Implications for ranking

This approximation motivates ranking bit positions (or weights) by:

* $\delta^2 \cdot H_{ii}$: bit-flip magnitude times curvature
* This is the FKeras method: estimates $H_{ii}$ and ranks accordingly
* You could generalize it to your method:
$\text{Impact} \cdot H$, not just gradients

---

## 5. When does this approximation hold?

?? Works well when:

* Bit-flip magnitude is small (i.e., local region)
* Model is near a minimum
* Hessian is stable (not exploding)

?? Fails when:

* Model isn??t trained well (gradient is large)
* Loss surface is highly non-quadratic

---

## Summary

The formula:
$\Delta \mathcal{L} \approx \frac{1}{2} \Delta \mathbf{w}^T H \Delta \mathbf{w}$

tells us how bit-flips propagate into loss increases, and explains why the Hessian is so powerful for ranking robustness. 
It encodes:

* How impactful a perturbation is (via $\delta$)
* How sensitive the loss is locally (via $H$)

"""

# Create markdown
md = pg.Markdown(md)

# Add to the first tab
tabmotivation.append(md)

""" Add quantization container to doc report """
tabpretraining.extend(Q.html())

""" Save doc to file (we save after adding each element) """
doc.save(filename)

# Let's create a container for all benchmarks 
benchmark_ct = pg.CollapsibleContainer("🧺 Benchmarks", layout='vertical')

# And another one for tabtraining
benchmark_sessions_ct = pg.CollapsibleContainer("🧺 Benchmarks", layout='vertical')

# And another one for tabposttraining
benchmark_posttraining_ct = pg.CollapsibleContainer("🧺 Benchmarks", layout='vertical')

# And yet another for "BER Injection"
benchmark_ber_ct = pg.CollapsibleContainer("🧪 Experiments", layout='vertical')

""" Add to documnt """
tabpretraining.append(benchmark_ct)

""" Add """
tabtraining.append(benchmark_sessions_ct)

""" Add """
tabposttraining.append(benchmark_posttraining_ct)

""" Add """
tabber.append(benchmark_ber_ct)

# Define benchmarks to analyze
#benchmarks = ['dummy', 'mnist_hls4ml', 'autompg', 'smartpixel_small', 'smartpixel_large',
#              'cifar10', 'mnist_lenet5', 'ECONT_AE'
# 'cifar100', 'svhn', 'fashion_mnist', 'imdb', 'reuters', 'boston_housing']
# TODO: Fix visualization/contrast for cifar10
# TODO: mnist_lenet5 seems to be working (good accuracy), but I'm not too happy about the alphas/betas. Some layers still have a big portion outside of the valid interval

config_per_methods = netsurf.config.config_per_method
protection_range = (0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4) #netsurf.config.DEFAULT_PROTECTION
ber_range = (0.001, 0.005, 0.01, 0.05, 0.1, 0.15) #netsurf.config.DEFAULT_BER

#methods = ['qpolar', 'qpolargrad', 'bitwise_msb', 'random', 'hirescam_norm', 
#           'hiresdelta', 'hessian', 'hessiandelta', 'weight_abs_value']
methods = ['laplace', 'hessian', 'fisher', 'bitwise_msb', 'random', 'grad_norm', 'graddelta', 'weight_abs_value'] # 'qpolar', 'qpolargrad', 
methods = methods

# Create benchmark object 
bmk = netsurf.get_benchmark(benchmark_name, Q,  benchmarks_dir = benchmarks_dir,
                            datasets_dir = datasets_dir, pruning = pruning,
                            load_weights = False)

# Add benchmark html to container (this includes model + dataset htmls)
# (run before training the model...)
benchmark_ct.append(bmk.html())

# Now let's prepare the data
nsample_mod = 48 if 'ECON' in bmk.name else -1
XYTrain = netsurf.utils.prepare_data(bmk, subset = 'train', nsample_mod = nsample_mod)

# # Initialize the uncertainty profiler (pre-training)
# pre_robustness_sgn_path = os.path.join(bmk.model_dir, 'uncertainty_signatures', f'{benchmark_name}.pretraining.netsurf.sgn')
# try:
#     pre_robustness_sgn = netsurf.UncertaintyProfiler.profile(bmk.model, XYTrain, bmk.model.loss, 
#                                                             batch_size = 2000, filepath = pre_robustness_sgn_path,
#                                                             verbose = True)
#     # Save 
#     pre_robustness_sgn.save_to_file(pre_robustness_sgn_path)
# except:
#     pass

# Add profile to tabpretraining
# benchmark_ct.append(pre_robustness_sgn.html())

# Now we can reload the weights (After the profiling)
bmk.load_weights(verbose = True)

# TRAINING - SESSION
# Try to get a session (if not, train)
sess = netsurf.get_training_session(bmk, show_plots = False, plot = True)


🧮 <QuantizationScheme(q<6,2,1>)> obj @ (0x7f5c10d2caf0):
    Total number of bits (m): 6
    Integer bits         (n): 2
    Float bits           (f): 3
    Signed
    Range: (-4, 3.875)
    Min step: 0.125
    Format: Sxx.xxx

7_HLS_q6_2_1_bmark_keyword_spotting_prune0_125.html


╭───────┼─────────────┤
╰ BMK ──┤ 10:39:51.21 │ - Initializing benchmark object keyword_spotting
        │ 10:39:51.21 │ - Loss sparse_categorical_crossentropy found in tf.keras.losses with definition <function sparse_categorical_crossentropy at 0x7f5930833130>.
        │ 10:39:51.21 │ - Adding custom metric sparse_categorical_accuracy with definition SparseCategoricalAccuracy(name=sparse_categorical_accuracy,dtype=float32).


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


        │ 10:39:56.56 │ - Benchmark object keyword_spotting initialized
╭───────┼─────────────┤
╰ PLOT ─┤ 10:40:18.46 │ - Variable qq_conv2d_4/kernel:0 has sparsity 0.00%
        │ 10:40:18.47 │ - Variable qq_conv2d_3/kernel:0 has sparsity 0.00%
        │ 10:40:18.48 │ - Variable qq_conv2d_1/kernel:0 has sparsity 0.00%
        │ 10:40:18.49 │ - Variable qq_conv2d_2/kernel:0 has sparsity 0.00%
        │ 10:40:18.51 │ - Variable qq_dense/kernel:0 has sparsity 0.00%
        │ 10:40:18.52 │ - Variable qq_conv2d/kernel:0 has sparsity 0.00%
        │ 10:40:18.52 │ - Variable qq_depthwise_conv2d_2/depthwise_kernel:0 has sparsity 0.00%
        │ 10:40:18.53 │ - Variable qq_depthwise_conv2d_3/depthwise_kernel:0 has sparsity 0.00%
        │ 10:40:18.54 │ - Variable qq_depthwise_conv2d/depthwise_kernel:0 has sparsity 0.00%
        │ 10:40:18.54 │ - Variable qq_depthwise_conv2d_1/depthwise_kernel:0 has sparsity 0.00%
        │ 10:40:18.55 │ - Variable qq_batch_normalization_6/gamma:0 has sparsity 

[WARN] - HTML not implemented for keyword_spotting


╭───────┼─────────────┤
╰ PLT ──┤ 10:40:35.47 │ - Plotting histogram for activation qq_conv2d
        │ 10:40:35.56 │ - Plotting histogram for activation qq_apply_alpha
        │ 10:40:35.65 │ - Plotting histogram for activation qq_batch_normalization
        │ 10:40:35.74 │ - Plotting histogram for activation qq_activation
        │ 10:40:35.84 │ - Plotting histogram for activation qq_dropout
        │ 10:40:35.93 │ - Plotting histogram for activation qq_depthwise_conv2d
        │ 10:40:36.02 │ - Plotting histogram for activation qq_apply_alpha_1
        │ 10:40:36.11 │ - Plotting histogram for activation qq_batch_normalization_1
        │ 10:40:36.20 │ - Plotting histogram for activation qq_activation_1
        │ 10:40:36.30 │ - Plotting histogram for activation qq_conv2d_1
        │ 10:40:36.39 │ - Plotting histogram for activation qq_apply_alpha_2
        │ 10:40:36.48 │ - Plotting histogram for activation qq_batch_normalization_2
        │ 10:40:36.57 │ - Plotting histogram for ac

Labels are not the same size as the confusion matrix


        │ 10:41:18.15 │ - Saved ROC plot to /mnt/raid0/asic/projects/NU/netsurf/manuelbv/benchmarks/keyword_spotting/q6_2_1/pruned_0.125_DSConv/sessions/training_session.20250416_000619/ROC.png
        │ 10:41:18.15 │ - Plotting confusion matrix for labels ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
        │ 10:41:18.49 │ - Saved confusion matrix to /mnt/raid0/asic/projects/NU/netsurf/manuelbv/benchmarks/keyword_spotting/q6_2_1/pruned_0.125_DSConv/sessions/training_session.20250416_000619/confusion_matrix.png


In [3]:
bmk.model.summary()

Model: "ds_conv"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 49, 10, 1)]       0         
                                                                 
 qq_conv2d (QQConv2D)        (None, 25, 5, 64)         52417     
                                                                 
 qq_apply_alpha (QQApplyAlph  (None, 25, 5, 64)        2434      
 a)                                                              
                                                                 
 qq_batch_normalization (QQB  (None, 25, 5, 64)        2625      
 atchNormalization)                                              
                                                                 
 qq_activation (QQActivation  (None, 25, 5, 64)        1         
 )                                                               
                                                           

In [ ]:
# Create a container for this bmk in tabtraining
bmk_sess_ct = pg.CollapsibleContainer(benchmark_name, layout='vertical')

# Add session to tabtraining
bmk_sess_ct.append(sess.html())

# And add to benchmarks in tabtraining
benchmark_sessions_ct.append(bmk_sess_ct)

# Add benchmark again to post-training to check how the weights changed
#benchmark_posttraining_ct.append(bmk.html())

# Initialize the uncertainty profiler (post-training)
# try:
#     post_robustness_sgn_path = os.path.join(bmk.model_dir, 'uncertainty_signatures', f'{benchmark_name}.posttraining.netsurf.sgn')
#     post_robustness_sgn = netsurf.UncertaintyProfiler.profile(bmk.model, XYTrain, bmk.model.loss, 
#                                     batch_size = 2000, verbose = True, filepath = post_robustness_sgn_path)

#     # Save 
#     post_robustness_sgn.save_to_file(post_robustness_sgn_path)
# except:
#     pass

# Add profile to tabposttraining
# benchmark_posttraining_ct.append(post_robustness_sgn.html())

# Let's compare the divergence between the profiles
# div_profile = netsurf.ProfileDivergence.from_signatures(pre_robustness_sgn, post_robustness_sgn)

#div_profile.plot_divergence_summary()
#div_profile.plot_advanced_divergence_summary()

""" Save doc to file (we save after adding each element) """
doc.save(filename)

# Create a container for this benchmark in "BER Injection"
bmk_ber_ct = pg.CollapsibleContainer(benchmark_name, layout='vertical')

# Add to tabber
benchmark_ber_ct.append(bmk_ber_ct)

# Create another container to store the individual results of the experiments
exp_individual_ct = pg.CollapsibleContainer("🎁 Individual results", layout='vertical')
bmk_ber_ct.append(exp_individual_ct)

# Create our ranker profiler (empty)
rankers_bucket = netsurf.RankingComparator()

# Loop thru methods
exps = {}
for method_alias in methods:
    #########################################################################
    # 0. Create the directory for the experiment (ranking, results, etc.)
    #########################################################################
    method = config_per_methods[method_alias]['method']
    method_dir = os.path.join(bmk.experiments_dir, method)

    #########################################################################
    # 1. Create the ranker
    #########################################################################
    rkr = rankers_bucket.build_ranker(method, Q, config = config_per_methods[method], 
                                    path = method_dir,
                                    is_baseline = (method == 'qpolargrad'))

    # Get the exp dir from the ranker (with the hash)
    exp_dir = rkr.path

    #################################################################
    # 1. Perform ranking according to method
    #################################################################
    # Rank weights 
    ranking = rkr.rank(bmk.model, *XYTrain, verbose = True)
    # Save rank to csv file 
    ranking.save(overwrite = False)

    #################################################################
    # 2. Create experiment object
    #################################################################
    exp = netsurf.Experiment(bmk, ranking, num_reps = 100, 
                            ber_range = ber_range, 
                            protection_range = protection_range, 
                            path = exp_dir,
                            verbose = True)

    # Print experiment info 
    print(exp)

    #################################################################
    # 3. Run experiment with given ranking and for whatever 
    #       range of protection and rad 
    #################################################################
    #batch_size = 1000,
    exp.run_experiment(bmk, XYTrain,
                    batch_size = None,
                    ber_range = ber_range, 
                    protection_range = protection_range, 
                    rerun = False)
    
    # Add experiment to container
    exp_individual_ct.append(exp.html())

    # Save experiment object
    exp.save()

    # Add to dict
    exps[method] = exp

    """ Save doc to file (we save after adding each element) """
    doc.save(filename)

""" Now perform the comparison of rankers """
#df = rankers_bucket.compare_rankers(granularity = 0.01, bootstrap = False)

# Add comparison to container
#bmk_ber_ct.append(rankers_bucket.html())

# Save doc 
doc.save(filename)

# # Save the rankers_bucket
# rankers_comparison_filepath = os.path.join(bmk.experiments_dir, 'ranking_comparison.csv')
# rankers_bucket.save_to_csv(rankers_comparison_filepath)

# """ Create Experiment Comparator """
# exp_comp = netsurf.ExperimentComparator(list(exps.values()))
#df = ExperimentComparator.compute_ranking_distribution(res)

In [ ]:
qqmodel = bmk.model
qqmodel.summary()

In [ ]:
# Let's now translate the model into a simple qkeras model for compilation
import qkeras
import tensorflow as tf

# Get the quantization scheme 
Q = qqmodel.quantizer
print(Q)
q_scheme = f'quantized_bits({Q.m},{Q.n},{bool(Q.s)}, alpha = 1, use_variables = True)'
q_relu = f'quantized_relu({Q.m},{Q.n},{bool(Q.s)}, use_variables = True)'
q_precision = f'ac_fixed<{Q.m},{Q.n+Q.s},{bool(Q.s)}>'
n2 = Q.n*2
f2 = Q.f*2
m2 = n2 + f2 + int(Q.s)
double_q_precision = f'ac_fixed<{m2},{n2+Q.s},{bool(Q.s)}>'

print(q_scheme)


In [ ]:
inputs = None 
for ilayer, layer in enumerate(qqmodel.layers):
    print(layer.name)
    name = layer.name.replace('qq_', '')
    if isinstance(layer, tf.keras.layers.InputLayer):
        x = tf.keras.layers.Input(shape = layer.input_shape[0][1:], name = 'in0')
        if inputs is None:
            inputs = [x]
        elif isinstance(inputs, list):
            inputs.append(x)

    elif isinstance(layer, tf.keras.layers.Flatten):
        x = tf.keras.layers.Flatten(name = name)(x)
    
    elif isinstance(layer, tf.keras.layers.Dense):
        ly = qkeras.QDense(layer.units, kernel_quantizer = q_scheme, name = name)
        
        # Pass the tensor so we can build the layer 
        x = ly(x)

        # The next one MUST be a qqapplyalpha
        next_layer = qqmodel.layers[ilayer + 1]
        if isinstance(next_layer, netsurf.dnn.layers.QQApplyAlpha):
            
            # Transfer the weights over
            # Remember we will have to apply the QAlpha to the dense layer before it
            ws_fc0 = {v.name.split('/')[1].split(':')[0]: v for v in ly.trainable_variables}
            ws_alpha0 = {v.name.split('/')[1].split(':')[0]: v for v in next_layer.trainable_variables}

            # merge fc0 and alpha0
            ws_fc0_alpha0 = {'kernel': ws_fc0['kernel']*ws_alpha0['alpha'], 'bias': ws_fc0['bias']*ws_alpha0['alpha'] + ws_alpha0['beta']}
            ly.kernel.assign(ws_fc0_alpha0['kernel'])
            ly.bias.assign(ws_fc0_alpha0['bias'])
        elif isinstance(next_layer, tf.keras.layers.Softmax) or isinstance(next_layer, tf.keras.layers.Activation) or isinstance(next_layer, netsurf.dnn.layers.QQSoftmax) or isinstance(next_layer, netsurf.dnn.layers.QQActivation):
            # It's okay
            continue
        else:
            raise ValueError(f"Layer {layer.name} should be followed by a QQApplyAlpha layer, but got {next_layer.name} instead")

    
    elif isinstance(layer, tf.keras.layers.Conv2D):
        ly = qkeras.QConv2D(layer.filters, kernel_size = layer.kernel_size, strides = layer.strides,
                        padding = layer.padding, kernel_quantizer = q_scheme, name = name)
        
        # Pass the tensor so we can build the layer 
        x = ly(x)

        # The next one MUST be a qqapplyalpha
        next_layer = qqmodel.layers[ilayer + 1]
        if isinstance(next_layer, netsurf.dnn.layers.QQApplyAlpha):
            
            # Transfer the weights over
            # Remember we will have to apply the QAlpha to the dense layer before it
            ws_fc0 = {v.name.split('/')[1].split(':')[0]: v for v in ly.trainable_variables}
            ws_alpha0 = {v.name.split('/')[1].split(':')[0]: v for v in next_layer.trainable_variables}

            # merge fc0 and alpha0
            ws_fc0_alpha0 = {'kernel': ws_fc0['kernel']*ws_alpha0['alpha'], 'bias': ws_fc0['bias']*ws_alpha0['alpha'] + ws_alpha0['beta']}
            ly.kernel.assign(ws_fc0_alpha0['kernel'])
            ly.bias.assign(ws_fc0_alpha0['bias'])
        
        elif isinstance(next_layer, tf.keras.layers.Softmax) or isinstance(next_layer, tf.keras.layers.Activation) or isinstance(next_layer, netsurf.dnn.layers.QQSoftmax) or isinstance(next_layer, netsurf.dnn.layers.QQActivation):
            # It's okay
            continue
        else:
            raise ValueError(f"Layer {layer.name} should be followed by a QQApplyAlpha layer, but got {next_layer.name} instead")

    
    elif isinstance(layer, tf.keras.layers.MaxPooling2D):
        x = tf.keras.layers.MaxPooling2D(pool_size = layer.pool_size, strides = layer.strides,
                                        padding = layer.padding, name = name)(x)
    
    elif isinstance(layer, tf.keras.layers.AveragePooling2D):
        x = tf.keras.layers.AveragePooling2D(pool_size = layer.pool_size, strides = layer.strides,
                                        padding = layer.padding, name = name)(x)
    
    elif isinstance(layer, tf.keras.layers.Activation) or isinstance(layer, netsurf.dnn.layers.QQActivation):
        x = qkeras.QActivation(layer.activation, name = name)(x)
    
    elif isinstance(layer, tf.keras.layers.Dropout):
        x = tf.keras.layers.Dropout(layer.rate, name = name)(x)
    
    elif isinstance(layer, tf.keras.layers.BatchNormalization):
        ly = tf.keras.layers.BatchNormalization(name = name)
        # Pass 
        x = ly(x)
        # We need to pass over the weights
        # Transfer the weights over
        # Remember we will have to apply the QAlpha to the dense layer before it
        ws_fc0 = {v.name.split('/')[1].split(':')[0].replace('qq_',''): v for v in layer.trainable_variables}
        # Add moving_variance and moving_average
        ws_fc0['moving_mean'] = layer.moving_mean
        ws_fc0['moving_variance'] = layer.moving_variance

        # merge fc0 and alpha0
        for key in ws_fc0.keys():
            if 'gamma' in key:
                print(f'Assigning layer.gamma to {key}')
                ly.gamma.assign(ws_fc0[key])
            elif 'beta' in key:
                print(f'Assigning layer.beta to {key}')
                ly.beta.assign(ws_fc0[key])
            elif 'moving_mean' in key:
                print(f'Assigning layer.moving_mean to {key}')
                ly.moving_mean.assign(ws_fc0[key])
            elif 'moving_variance' in key:
                print(f'Assigning layer.moving_variance to {key}')
                ly.moving_variance.assign(ws_fc0[key])
            else:
                raise ValueError(f"Unknown key {key} in BatchNormalization layer {layer.name}")

    
    elif isinstance(layer, tf.keras.layers.Softmax):
        x = tf.keras.layers.Softmax(name = name)(x)
    else:
        if not isinstance(layer, netsurf.dnn.layers.QQApplyAlpha):
            print(f"Layer {name} not supported, skipping...")
        continue

# Build model
model = tf.keras.models.Model(inputs=inputs, outputs=x)
model.summary()

In [ ]:
# Loop thru weights to check their values 
import matplotlib.pyplot as plt 
for v in model.trainable_variables:
    plt.hist(v.numpy().flatten(), bins=100)
    plt.title(v.name)
    plt.show()

In [ ]:
benchmarks_dir = os.path.join(parent_dir, 'benchmarks')
datasets_dir = os.path.join(parent_dir, 'datasets')


In [ ]:
# Then the QKeras model
import hls4ml

outpath = os.path.join(bmk.models_dir,'syn')
os.makedirs(outpath, exist_ok = True)

# Convert model to hls4ml
hls_config_q = hls4ml.utils.config_from_keras_model(model, granularity='name')
hls_config_q['Model']['ReuseFactor'] = 2000
hls_config_q['Model']['Precision'] = q_precision
hls_config_q["Model"]["Strategy"] = "Resource"
hls_config_q["Model"]["BramFactor"] = 0

"""
    Input
"""
hls_config_q['LayerName']['in0']['Precision']['result'] = 'ac_fixed<8,1>'


# Loop thru layers and assign precision
for ly in model.layers:
    # Get name 
    name = ly.name 
    if isinstance(ly, tf.keras.layers.Dense) or isinstance(ly, qkeras.QDense) or isinstance(ly, tf.keras.layers.Conv2D) or isinstance(ly, qkeras.QConv2D):
        hls_config_q['LayerName'][name]['Precision']['weight'] = q_precision
        hls_config_q['LayerName'][name]['Precision']['bias'] = q_precision
        hls_config_q['LayerName'][name]['Precision']['result'] = double_q_precision
        hls_config_q['LayerName'][name]['ReuseFactor'] = 2000
    
    elif isinstance(ly, tf.keras.layers.BatchNormalization):
        hls_config_q['LayerName'][name]['Precision']['scale'] = q_precision
        hls_config_q['LayerName'][name]['Precision']['bias'] = q_precision
        hls_config_q['LayerName'][name]['Precision']['result'] = double_q_precision
        hls_config_q['LayerName'][name]['Precision']['mean'] = q_precision
        hls_config_q['LayerName'][name]['Precision']['variance'] = q_precision
        hls_config_q['LayerName'][name]['Precision']['moving_variance'] = q_precision
        hls_config_q['LayerName'][name]['Precision']['moving_mean'] = q_precision
    
    elif isinstance(ly, tf.keras.layers.Softmax):
        hls_config_q['LayerName'][name]['Strategy'] = 'Stable'
        hls_config_q['LayerName']['softmax']['Precision'] = f'ac_fixed<{Q.m},0,false>'
        hls_config_q['LayerName']['softmax']['exp_table_t'] = 'ac_fixed<4,0>'
        hls_config_q['LayerName']['softmax']['inv_table_t'] = 'ac_fixed<4,0>'
    
    else:
        # Just set the precision to be q_precision for the result 
        hls_config_q['LayerName'][name]['Precision']['result'] = q_precision









# """
#     fc0
# """
# # Weights & bias 
# hls_config_q['LayerName']['fc0']['Precision']['weight'] = 'ac_fixed<8,0,true>'
# # Set bias precision
# hls_config_q['LayerName']['fc0']['Precision']['bias'] = q_precision
# # Set activation precision
# hls_config_q['LayerName']['fc0']['Precision']['result'] = double_q_precision
# hls_config_q['LayerName']['fc0']['FlattenWeights'] = True
# hls_config_q['LayerName']['fc0']['ReuseFactor'] = 2000

# """
#     fcout
# """
# # Weights & bias 
# hls_config_q['LayerName']['fc_out']['Precision']['weight'] = q_precision
# # Set bias precision
# hls_config_q['LayerName']['fc_out']['Precision']['bias'] = q_precision
# # Set activation precision
# hls_config_q['LayerName']['fc_out']['Precision']['result'] = double_q_precision
# hls_config_q['LayerName']['fc_out']['ReuseFactor'] = 640


# # fcout
# hls_config_q['LayerName']['fc_out']['Precision']['result'] = "ac_fixed<8,1,false>"

# """ 
#     softmax
# """
# hls_config_q['LayerName']['softmax']['Strategy'] = 'Stable'
# hls_config_q['LayerName']['softmax']['Precision'] = f'ac_fixed<{Q.m},0,false>'
# hls_config_q['LayerName']['softmax']['exp_table_t'] = 'ac_fixed<4,0>'
# hls_config_q['LayerName']['softmax']['inv_table_t'] = 'ac_fixed<4,0>'

import json 
print(json.dumps(hls_config_q, indent=4))


In [ ]:

""" Create catapult config """
cfg_q = hls4ml.converters.create_config(backend='catapult')
cfg_q['IOType'] = 'io_stream'  # Must set this if using CNNs!
cfg_q['HLSConfig'] = hls_config_q
cfg_q['KerasModel'] = model
cfg_q['OutputDir'] = outpath
#cfg_q['Technology'] = 'fpga' # inportant, should be in lower case
cfg_q['Technology'] = 'asic' # inportant, should be in lower case
cfg_q['ClockPeriod'] = 10
cfg_q['ASICLibs'] = 'nangate-45nm_beh'
cfg_q['FIFO'] = 'hls4ml_lib.mgc_pipe_mem'
cfg_q['ASICFIFO']='hls4ml_lib.mgc_pipe_mem'
#cfg_q['XilinxPart'] = 'xczu7cg-fbvb900-1-e'
#cfg_q['Part'] = 'xczu7cg-fbvb900-1-e'
#cfg_q.pop['XilinxPart']


# cfg_q['BuildOptions'] = {
#             'csim': True,
#             'SCVerify': True,
#             'Synth': True,
#             'vhdl': False,
#             'verilog': True,
#             'RTLSynth': False,
#             'RandomTBFrames': 2,
#             'PowerEst': False,
#             'PowerOpt': False,
#             'BuildBUP': False,
#             'BUPWorkers': False,
#             'LaunchDA': False,
#         }

print(cfg_q)

hls_model_q = hls4ml.converters.keras_to_hls(cfg_q)
hls_model_q.compile()